In [2]:
from optimum.onnxruntime import ORTModelForSeq2SeqLM
from transformers import AutoTokenizer
from pathlib import Path

model_name = "digit82/kobart-summarization"
onnx_dir = Path("../../automation/models/kobart_summary_onnx")

# ✅ 1. 모델 변환 (export=True) ← 저장 안됨
model = ORTModelForSeq2SeqLM.from_pretrained(model_name, export=True)

# ✅ 2. ONNX 모델 저장
model.save_pretrained(onnx_dir)

# ✅ 3. 토크나이저 저장
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained(onnx_dir)

print("🎉 ONNX 변환 및 저장 완료:", onnx_dir)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
c:\Users\user\anaconda3\envs\test-0602\lib\site-packages\transformers\models\bart\

🎉 ONNX 변환 및 저장 완료: ..\..\automation\models\kobart_summary_onnx


In [16]:
from optimum.onnxruntime import ORTModelForSeq2SeqLM
from tokenizers import Tokenizer
from pathlib import Path
import torch

def get_summarize_model():
    """
    ONNX 모델과 토크나이저를 반환하는 함수
    """

    model_dir = Path("../../automation/models/kobart_summary_onnx")
    model_summarize = ORTModelForSeq2SeqLM.from_pretrained(model_dir, local_files_only=True)

    tokenizer_summarize = Tokenizer.from_file(str(model_dir / "tokenizer.json"))

    return model_summarize, tokenizer_summarize

c:\Users\user\anaconda3\envs\test-0602\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import numpy as np
import onnxruntime as ort
from tokenizers import Tokenizer
from pathlib import Path

# ⬇ 모델 세션 초기화
def load_model():
    base_path = Path("../../automation/models/kobart_summary_onnx")
    encoder_sess = ort.InferenceSession(str(base_path / "encoder_model.onnx"))
    decoder_sess = ort.InferenceSession(str(base_path / "decoder_model.onnx"))
    tokenizer = Tokenizer.from_file(str(base_path / "tokenizer.json"))
    return encoder_sess, decoder_sess, tokenizer


# ⬇ 디코딩 함수 (greedy 또는 beam=1)
def summarize_event_focused(
    text,
    encoder_sess,
    decoder_sess,
    tokenizer,
    max_length=128,
    no_repeat_ngram_size=3,
    repetition_penalty=1.2,
):
    input_ids = tokenizer.encode(text).ids
    input_ids_np = np.array([input_ids], dtype=np.int64)
    attention_mask = np.ones_like(input_ids_np, dtype=np.int64)

    encoder_outputs = encoder_sess.run(
        None, {"input_ids": input_ids_np, "attention_mask": attention_mask}
    )[0]

    decoder_input_ids = [tokenizer.token_to_id("<s>")]
    generated_ids = decoder_input_ids.copy()

    for _ in range(max_length):
        decoder_input_np = np.array([generated_ids], dtype=np.int64)
        decoder_inputs = {
            "input_ids": decoder_input_np,
            "encoder_hidden_states": encoder_outputs,
            "encoder_attention_mask": attention_mask,
        }
        logits = decoder_sess.run(None, decoder_inputs)[0]
        next_token_logits = logits[:, -1, :]

        # repetition penalty 적용
        for token_id in set(generated_ids):
            next_token_logits[0, token_id] /= repetition_penalty

        # no_repeat_ngram_size 적용
        if no_repeat_ngram_size > 0 and len(generated_ids) >= no_repeat_ngram_size:
            ngram = tuple(generated_ids[-(no_repeat_ngram_size - 1) :])
            banned = {
                tuple(generated_ids[i : i + no_repeat_ngram_size])
                for i in range(len(generated_ids) - no_repeat_ngram_size + 1)
            }
            for token_id in range(next_token_logits.shape[-1]):
                if ngram + (token_id,) in banned:
                    next_token_logits[0, token_id] = -1e9  # 큰 마이너스

        # greedy 선택
        next_token_id = int(np.argmax(next_token_logits, axis=-1)[0])

        if next_token_id == tokenizer.token_to_id("</s>"):
            break

        generated_ids.append(next_token_id)

    return tokenizer.decode(generated_ids, skip_special_tokens=True)

In [37]:
encoder_sess, decoder_sess, tokenizer = load_model()

In [58]:
# 입력 문장
text = """
'사진=한경DB\n키움증권은 23일 삼성바이오로직스에 대해 인적분할을 추진하는 건 본격적인 신약 개발에 나서겠다는 의미하고 해석했다.\n삼성바이오로직스는 인적분할을 통해 의약품 위탁 개발·생산(CDMO) 부문은 존속법인에 남기고, 신설법인인 삼성에피스홀딩스에 삼성바이오에피스 지분 100%를 승계하도록 하는 방안을 추진하겠다고 공시했다.\n이에 대해 허혜민 키움증권 연구원은 “삼성그룹이 (삼성에피스홀딩스를 통해) 본격적으로 신약 개발을 시작한다는 의미로 해석된다”고 말했다. 이어 “삼성바이오로직스는 글로벌 파트너십 확대를 위한 영업활동이 수월해질 것”이라며 “분할된 신설법인은 투자지주회사로 신성장 동력 발굴과 연구·개발(R&D) 및 기업 인수·합병(M&A)를 통해 적극적인 성장을 추진할 것”이라고 내다봤다.\n이번 분할 추진이 단기적으로 펀더멘털에 미치는 영향은 미미할 것으로 분석됐다.\n다만 허 연구원은 “최근 인적분할 업체들의 분할상장 전 주가 수익률이 대체로 상승 흐름을 나타냈다”며 “이후 각 사의 펀더멘털, 수급, 모멘텀에 따라 변동했다”고 전했다.
"""

summary = summarize_event_focused(text, encoder_sess, decoder_sess, tokenizer)

print("📄 요약:", summary)

📄 요약: 삼성바이오로직스는 의약품 위탁 개발·생산(CDMO) 부문은 존속법인에 남기고, 신설법인인 삼성바이오에피스 지분 100%를 승계하도록 하는 방안을 추진하겠다고 밝히며 본격적인 신약 개발에 나서겠다는 의미하고 해석하였다.


In [29]:
import pandas as pd

# 행 전체 출력
pd.set_option("display.max_rows", None)

# 열 전체 출력
pd.set_option("display.max_columns", 200)

# 각 열 너비 무제한 출력
pd.set_option("display.max_colwidth", 200)

# 전체 넓이 제한 해제
pd.set_option("display.width", None)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel


class SBERTMeanPooling(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(model_name)

    def forward(self, input_ids, attention_mask):
        output = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden = output.last_hidden_state  # [B, T, H]
        mask = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()
        summed = torch.sum(last_hidden * mask, dim=1)
        counts = torch.clamp(mask.sum(1), min=1e-9)
        mean_pooled = summed / counts  # [B, H]

        # ✅ L2 정규화 추가 (중요!)
        normalized = F.normalize(mean_pooled, p=2, dim=1)
        return normalized

c:\Users\user\anaconda3\envs\test-0602\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# export_sbert_mean_onnx.py

from transformers import AutoTokenizer
from pathlib import Path
import torch

# 설정
model_name = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"
onnx_path = Path("../../modelapi/models/kr_sbert_mean_onnx")
onnx_path.mkdir(parents=True, exist_ok=True)

# 모델 및 토크나이저 로딩
model = SBERTMeanPooling(model_name).eval()
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 더미 입력 생성
inputs = tokenizer(
    ["이 문장은 테스트입니다."], return_tensors="pt", padding=True, truncation=True
)
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# ONNX export
torch.onnx.export(
    model,
    (input_ids, attention_mask),
    str(onnx_path / "kr_sbert.onnx"),
    input_names=["input_ids", "attention_mask"],
    output_names=["sentence_embedding"],
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "sequence_length"},
        "attention_mask": {0: "batch_size", 1: "sequence_length"},
        "sentence_embedding": {0: "batch_size"},
    },
    opset_version=14,
)

# tokenizer 저장
tokenizer.backend_tokenizer.save(str(onnx_path / "tokenizer.json"))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [3]:
import onnxruntime as ort
from tokenizers import Tokenizer
from pathlib import Path


def get_kr_sbert_model():
    base_path = Path("../../modelapi/models/kr_sbert_mean_onnx")

    tokenizer = Tokenizer.from_file(str(base_path / "tokenizer.json"))
    session = ort.InferenceSession(str(base_path / "kr_sbert.onnx"))

    return tokenizer, session

In [4]:
import numpy as np

tokenizer, session = get_kr_sbert_model()

# 입력 텍스트
text = "이 문장은 테스트입니다."
tokens = tokenizer.encode(text)
input_ids = np.array([tokens.ids], dtype=np.int64)
attention_mask = np.array([[1] * len(tokens.ids)], dtype=np.int64)

# 추론
embedding = session.run(
    ["sentence_embedding"], {"input_ids": input_ids, "attention_mask": attention_mask}
)[0]

print("문장 임베딩 벡터:", embedding.shape)  # ✅ (1, 768)

문장 임베딩 벡터: (1, 768)


In [1]:
import requests

url = "http://15.165.211.100:8000/models/embedding"  # 예: http://localhost:8000/models/embedding
headers = {"Content-Type": "application/json"}

In [2]:
data = {
    "article": "삼성전자의 임원이 대거 탈출했습니다."
}

response1 = requests.post(url, json=data, headers=headers)

print("Status Code:", response1.status_code)
print("Response:", response1.json())

embedding1 = response1.json().get("embedding")

ConnectionError: HTTPConnectionPool(host='15.165.211.100', port=8000): Max retries exceeded with url: /models/embedding (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10809f220>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [29]:
data = {
    "article": "SK하이닉스가가 새로운 AI 반도체를 발표했습니다. 주가에 긍정적인 영향을 줄 수 있습니다."
}

response2 = requests.post(url, json=data, headers=headers)

print("Status Code:", response2.status_code)
print("Response:", response2.json())

embedding2 = response2.json().get("embedding")

Status Code: 200
Response: {'embedding': [[-0.6082009077072144, -0.24337103962898254, -1.658624529838562, 0.08471480756998062, -0.38637271523475647, 0.25139158964157104, -0.4687521457672119, 0.3226619064807892, 0.8840901851654053, -0.6541084051132202, 0.05281134694814682, 1.675542950630188, 0.2493857443332672, 0.644999086856842, -0.46177351474761963, -0.22420170903205872, 1.2425320148468018, -1.4105069637298584, -1.1711419820785522, 1.2907905578613281, 0.03843836486339569, -0.7215048670768738, 0.33660221099853516, 0.6304286122322083, -0.9233324527740479, -0.2846764922142029, -0.8218221068382263, 0.3201751708984375, 0.03762371838092804, 1.2150095701217651, -0.4178465008735657, 0.18083268404006958, 1.1286430358886719, -1.4378163814544678, 0.7738139033317566, 0.037001676857471466, -0.4827357828617096, -0.2697122395038605, 0.42217427492141724, -0.308170348405838, -1.1753606796264648, 0.5722053647041321, -0.038359276950359344, -0.06039369851350784, 0.003216959536075592, -0.17520558834075928

In [27]:
from sklearn.metrics.pairwise import cosine_similarity


# numpy 변환 및 유사도 계산
embedding1_np = np.array(embedding1)
embedding2_np = np.array(embedding2)

cos_sim = cosine_similarity(embedding1_np, embedding2_np)

print("✅ Cosine Similarity:", cos_sim[0][0])

✅ Cosine Similarity: 1.0000000000000013


In [16]:
embedding

[[-0.6082009077072144,
  -0.24337103962898254,
  -1.658624529838562,
  0.08471480756998062,
  -0.38637271523475647,
  0.25139158964157104,
  -0.4687521457672119,
  0.3226619064807892,
  0.8840901851654053,
  -0.6541084051132202,
  0.05281134694814682,
  1.675542950630188,
  0.2493857443332672,
  0.644999086856842,
  -0.46177351474761963,
  -0.22420170903205872,
  1.2425320148468018,
  -1.4105069637298584,
  -1.1711419820785522,
  1.2907905578613281,
  0.03843836486339569,
  -0.7215048670768738,
  0.33660221099853516,
  0.6304286122322083,
  -0.9233324527740479,
  -0.2846764922142029,
  -0.8218221068382263,
  0.3201751708984375,
  0.03762371838092804,
  1.2150095701217651,
  -0.4178465008735657,
  0.18083268404006958,
  1.1286430358886719,
  -1.4378163814544678,
  0.7738139033317566,
  0.037001676857471466,
  -0.4827357828617096,
  -0.2697122395038605,
  0.42217427492141724,
  -0.308170348405838,
  -1.1753606796264648,
  0.5722053647041321,
  -0.038359276950359344,
  -0.0603936985135078

In [18]:
import re
from kss import split_sentences

def remove_market_related_sentences(text: str) -> str:
    # 줄바꿈 제거
    text = text.replace("\n", " ")

    # 대괄호 포함 텍스트 제거: [파이낸셜뉴스], [사진] 등
    text = re.sub(r"\[[^\]]*\]", "", text)

    # '/사진', '/사진제공' 제거
    text = re.sub(r"/사진(제공)?", "", text)

    # 이메일 주소 제거 (예: josh@yna.co.kr)
    text = re.sub(r"\b[\w.-]+@[\w.-]+\.\w+\b", "", text)

    # 문장 단위 분리 (간단하게 마침표 기준, 필요시 KSS 등 적용 가능)
    sentences = split_sentences(text)

    # 제거할 패턴들 (뉴스 문장에서 자주 등장하는 패턴)
    patterns = [
        r"(자세한 내용|자세한 사항)",  # 뉴스 기본 표현
        r"\d{4}[.-]\d{1,2}[.-]\d{1,2}",  # 날짜 (예: 2025.03.26, 2024-12-01)
        r"([0-9,]+(?:만)?[0-9,]*\s?(?:원|만원))",  # 가격 (예: 3,500원, 12000원)
        r"(강세|펀드|시가총액|등락률|한국거래소)",  # 증시 용어
        r"\([+-]?[0-9.,]+%\)",  # 괄호 안 퍼센트 등락률
        r"(투자의견|연구원|평가|예상치|증권가|리포트|팀장)",  # 애널리스트 용어
        r"(순이익|전년|매출|영업이익|영업적자|증시|코스피|코스닥|다우|나스닥|매출액|거래일|호조세|레버리지|투자자|조정|자산|수익률|이익률|수익성|내리막|부진한|낙폭|기대치|실적발표|기업 가치)",  # 시장 용어
    ]

    # 하나의 통합 패턴으로 컴파일
    combined_pattern = re.compile("|".join(patterns))

    # 필터링된 문장만 유지
    filtered = [s for s in sentences if not combined_pattern.search(s)]

    text_preprocessed = " ".join(filtered)

    # print(f"원문:{sentences}\n|\n전처리 된 문장: {text_preprocessed}\n\n")

    return text_preprocessed

In [19]:
df_5 = df.head(5).copy()
df_5['article'] = df_5['article'].apply(remove_market_related_sentences)

[Kss]: Oh! You have mecab in your environment. Kss will take this as a backend! :D



In [39]:
df_5['article'][2]

NameError: name 'df_5' is not defined

In [ ]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForTokenClassification
from pathlib import Path

model_id = "KPF/KPF-BERT-NER"
save_dir = Path("../../automation/models/ner_onnx")

# ONNX로 export
model = ORTModelForTokenClassification.from_pretrained(model_id, export=True)
model.save_pretrained(save_dir)

# tokenizer도 저장
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.save_pretrained(save_dir)

print("✅ NER ONNX 모델 저장 완료:", save_dir)

c:\Users\user\anaconda3\envs\test-0602\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ NER ONNX 모델 저장 완료: ..\..\automoation\models\ner_onnx


In [7]:
import onnxruntime as ort
from tokenizers import Tokenizer

def get_ner_tokenizer():
    """
    ONNX NER 모델을 위한 토크나이저를 반환합니다.
    """

    # 🟢 토크나이저 로딩
    tokenizer_ner = Tokenizer.from_file("../../automation/models/ner_onnx/tokenizer.json")

    # 🟢 ONNX 모델 세션 로딩
    session_ner = ort.InferenceSession("../../automation/models/ner_onnx/model.onnx")

    return tokenizer_ner, session_ner

In [8]:
import numpy as np
from label_map import id2label  # 라벨 ID ↔ 라벨명 매핑

tokenizer_ner, session_ner = get_ner_tokenizer()

def get_ner_tokens(tokenizer, session, text, id2label):
    # 🟡 토큰화 및 입력값 준비
    encoding = tokenizer.encode(text)
    input_ids = np.array([encoding.ids], dtype=np.int64)
    attention_mask = np.ones_like(input_ids, dtype=np.int64)
    token_type_ids = np.zeros_like(input_ids, dtype=np.int64)

    # 🔵 ONNX 추론 실행
    inputs = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "token_type_ids": token_type_ids,
    }

    logits = session.run(None, inputs)[0]  # shape: (1, seq_len, num_labels)

    # 🔵 라벨 인덱스 → 실제 라벨명
    preds = np.argmax(logits, axis=-1)[0]
    labels = [id2label[p] for p in preds[: len(encoding.tokens)]]

    # 🔵 시각화
    tokens = encoding.tokens

    return tokens, labels

In [9]:
def extract_ogg_economy(tokens, labels, target_label="OGG_ECONOMY"):
    merged_words = []
    current_word = ""

    for token, label in zip(tokens, labels):
        token_clean = token.replace("##", "") if token.startswith("##") else token

        if label == f"B-{target_label}":
            if current_word:
                merged_words.append(current_word)
            current_word = token_clean

        elif label == f"I-{target_label}":
            current_word += token_clean

        else:
            if current_word:
                merged_words.append(current_word)
                current_word = ""

    if current_word:
        merged_words.append(current_word)

    stock_list = merged_words.copy()

    return stock_list

In [22]:
import numpy as np
from label_map import id2label  # 라벨 ID ↔ 라벨명 매핑

text = """
세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자(COO)인 존 월드론이 방한해 이창용 한국은행 총재, 이재용 삼성전자 회장, 김병주 MBK파트너스 회장 등과 함께 한국의 금융당국, 기업 및 금융계와의 협력 방안을 논의했다.
"""

tokens, labels = get_ner_tokens(tokenizer_ner, session_ner, text, id2label)
stock_list = extract_ogg_economy(tokens, labels, target_label="OGG_ECONOMY")
stock_list

['골드만삭스', '한국은행', '삼성전자', 'MBK파트너스']

In [23]:
import pandas as pd

In [14]:
def load_official_stock_list(krx_csv_path):
    df = pd.read_csv(krx_csv_path, encoding="cp949")
    return set(df["종목명"].dropna().unique())

In [24]:
official_stock_set = load_official_stock_list("../../db/KRX_KOSPI.csv")

In [25]:
def filter_official_stocks_from_list(stock_list, official_stock_set):
    return [stock for stock in stock_list if stock in official_stock_set]

In [26]:
stock_list = filter_official_stocks_from_list(
    stock_list, official_stock_set
)

In [27]:
stock_list

['삼성전자']

In [43]:
summarzied_news = [
    {
        "news_id": "20250610_08827802",
        "wdate": "2025-06-10 17:45",
        "title": "컴포즈커피 산 필리핀 기업, 노랑통닭도 삼켰다",
        "article": "글로벌 외식업체 졸리비\n인수 우선협상대상자 선정\n연이은 M&A … 韓공략 속도\n필리핀 최대 외식업체 졸리비푸드(이하 졸리비)가 지난해 국내 저가 커피 2위 업체인 컴포즈커피를 인수한 데 이어 올해 하반기 국내 치킨 프랜차이즈 노랑통닭을 운용하는 노랑푸드도 인수할 예정이다.\n10일 투자은행(IB) 업계에 따르면 코스톤아시아와 큐캐피탈파트너스는 노랑푸드 인수 우선협상대상자로 졸리비를 선정하고 이날 오전 양해각서(MOU)를 체결했다.\n인수 주체는 졸리비의 한국법인인 졸리케이이며 조만간 상세 실사에 착수할 예정이다.\n양측은 오는 8월 말까지 실사를 마무리하고 올해 내로 주식매매계약(SPA)을 체결할 전망이다. 예상 매각가는 1000억원대 중반이다.\n졸리비는 큐캐피탈과 코스톤이 보유한 노랑통닭 운영사 노랑푸드 지분 100%를 인수하게 된다. 졸리비는 필리핀에 본사를 둔 글로벌 외식기업이다. 동남아시아를 비롯해 미국·유럽·중국 등 전 세계에서 매장을 운영 중이다.\n졸리비는 국내 사모펀드 운용사인 엘리베이션에쿼티파트너스와 함께 지난해 국내 저가 커피 2위 업체인 컴포즈커피를 4700억원에 인수하며 한국 시장에 처음으로 진출한 바 있다.\n코스톤과 큐캐피탈은 2020년 노랑푸드 지분 100%를 약 700억원에 인수했다.\n두 운용사가 인수한 이후 노랑통닭 실적은 크게 개선됐다. 2019년 말 400여 개였던 가맹점 수는 올해 700개를 넘겼다. 2019년 502억원이던 매출도 2024년 1067억원으로 두 배 넘게 올랐다.\n[나현준 기자 / 정슬기 기자]",
        "press": "매일경제",
        "url": "https://n.news.naver.com/mnews/article/009/0005506513",
        "image": "https://imgnews.pstatic.net/image/009/2025/06/10/0005506513_001_20250610174509011.jpg?type=w800",
        "article_preprocessed": "글로벌 외식업체 졸리비 인수 우선협상대상자 선정 연이은 M&A … 韓공략 속도 필리핀 최대 외식업체 졸리비푸드(이하 졸리비)가 지난해 국내 저가 커피 2위 업체인 컴포즈커피를 인수한 데 이어 올해 하반기 국내 치킨 프랜차이즈 노랑통닭을 운용하는 노랑푸드도 인수할 예정이다. 10일 투자은행(IB) 업계에 따르면 코스톤아시아와 큐캐피탈파트너스는 노랑푸드 인수 우선협상대상자로 졸리비를 선정하고 이날 오전 양해각서(MOU)를 체결했다. 인수 주체는 졸리비의 한국법인인 졸리케이이며 조만간 상세 실사에 착수할 예정이다. 양측은 오는 8월 말까지 실사를 마무리하고 올해 내로 주식매매계약(SPA)을 체결할 전망이다. 예상 매각가는 1000억원대 중반이다. 졸리비는 큐캐피탈과 코스톤이 보유한 노랑통닭 운영사 노랑푸드 지분 100%를 인수하게 된다. 졸리비는 필리핀에 본사를 둔 글로벌 외식기업이다. 동남아시아를 비롯해 미국·유럽·중국 등 전 세계에서 매장을 운영 중이다. 코스톤과 큐캐피탈은 2020년 노랑푸드 지분 100%를 약 700억원에 인수했다. 두 운용사가 인수한 이후 노랑통닭 실적은 크게 개선됐다. 2019년 말 400여 개였던 가맹점 수는 올해 700개를 넘겼다.",
        "summary": "필리핀 최대 외식업체 졸리비푸드가 국내 저가 커피 2위 업체인 컴포즈커피를 인수한 데 이어 올해 하반기 국내 치킨 프랜차이즈 노랑통닭을 운용하는 노랑푸드도 인수할 예정이다.",
    },
    {
        "news_id": "20250610_45742590",
        "wdate": "2025-06-10 17:44",
        "title": "컴포즈커피 산 필리핀 기업, 노랑통닭도 삼켰다",
        "article": "글로벌 외식업체 졸리비\n인수 우선협상대상자 선정\n연이은 M&A … 韓공략 속도\n필리핀 최대 외식업체 졸리비푸드(이하 졸리비)가 지난해 국내 저가 커피 2위 업체인 컴포즈커피를 인수한 데 이어 올해 하반기 국내 치킨 프랜차이즈 노랑통닭을 운용하는 노랑푸드도 인수할 예정이다.\n10일 투자은행(IB) 업계에 따르면 코스톤아시아와 큐캐피탈파트너스는 노랑푸드 인수 우선협상대상자로 졸리비를 선정하고 이날 오전 양해각서(MOU)를 체결했다.\n인수 주체는 졸리비의 한국법인인 졸리케이이며 조만간 상세 실사에 착수할 예정이다.\n양측은 오는 8월 말까지 실사를 마무리하고 올해 내로 주식매매계약(SPA)을 체결할 전망이다. 예상 매각가는 1000억원대 중반이다.\n졸리비는 큐캐피탈과 코스톤이 보유한 노랑통닭 운영사 노랑푸드 지분 100%를 인수하게 된다. 졸리비는 필리핀에 본사를 둔 글로벌 외식기업이다. 동남아시아를 비롯해 미국·유럽·중국 등 전 세계에서 매장을 운영 중이다.\n졸리비는 국내 사모펀드 운용사인 엘리베이션에쿼티파트너스와 함께 지난해 국내 저가 커피 2위 업체인 컴포즈커피를 4700억원에 인수하며 한국 시장에 처음으로 진출한 바 있다.\n코스톤과 큐캐피탈은 2020년 노랑푸드 지분 100%를 약 700억원에 인수했다.\n두 운용사가 인수한 이후 노랑통닭 실적은 크게 개선됐다. 2019년 말 400여 개였던 가맹점 수는 올해 700개를 넘겼다. 2019년 502억원이던 매출도 2024년 1067억원으로 두 배 넘게 올랐다.\n[나현준 기자 / 정슬기 기자]",
        "press": "한국경제",
        "url": "https://n.news.naver.com/mnews/article/009/0005506513",
        "image": "https://imgnews.pstatic.net/image/009/2025/06/10/0005506513_001_20250610174509011.jpg?type=w800",
        "article_preprocessed": "글로벌 외식업체 졸리비 인수 우선협상대상자 선정 연이은 M&A … 韓공략 속도 필리핀 최대 외식업체 졸리비푸드(이하 졸리비)가 지난해 국내 저가 커피 2위 업체인 컴포즈커피를 인수한 데 이어 올해 하반기 국내 치킨 프랜차이즈 노랑통닭을 운용하는 노랑푸드도 인수할 예정이다. 10일 투자은행(IB) 업계에 따르면 코스톤아시아와 큐캐피탈파트너스는 노랑푸드 인수 우선협상대상자로 졸리비를 선정하고 이날 오전 양해각서(MOU)를 체결했다. 인수 주체는 졸리비의 한국법인인 졸리케이이며 조만간 상세 실사에 착수할 예정이다. 양측은 오는 8월 말까지 실사를 마무리하고 올해 내로 주식매매계약(SPA)을 체결할 전망이다. 예상 매각가는 1000억원대 중반이다. 졸리비는 큐캐피탈과 코스톤이 보유한 노랑통닭 운영사 노랑푸드 지분 100%를 인수하게 된다. 졸리비는 필리핀에 본사를 둔 글로벌 외식기업이다. 동남아시아를 비롯해 미국·유럽·중국 등 전 세계에서 매장을 운영 중이다. 코스톤과 큐캐피탈은 2020년 노랑푸드 지분 100%를 약 700억원에 인수했다. 두 운용사가 인수한 이후 노랑통닭 실적은 크게 개선됐다. 2019년 말 400여 개였던 가맹점 수는 올해 700개를 넘겼다.",
        "summary": "필리핀 최대 외식업체 졸리비푸드가 국내 저가 커피 2위 업체인 컴포즈커피를 인수한 데 이어 올해 하반기 국내 치킨 프랜차이즈 노랑통닭을 운용하는 노랑푸드도 인수할 예정이다.",
    },
    {
        "news_id": "20250610_01590130",
        "wdate": "2025-06-10 17:44",
        "title": "한투證 '독점 골드만 리서치' 서학개미 투자 교과서 됐다",
        "article": "한달새 조회수 14만회 돌파\n김성환 한투證 사장\n한국투자증권이 지난달 골드만삭스와 제휴해 선보인 '독점 미국 현지 리서치'의 누적 조회수가 14만회를 돌파했다고 10일 밝혔다. 한 달간 발간된 보고서 가운데 서학개미들이 가장 주목한 이슈는 '3분기 중국 경제전망'이었다.\n'독점 미국 현지 리서치'는 한국투자증권이 글로벌 리서치 차별화를 위해 세계 최대 투자은행 골드만삭스와 손잡고 출시한 서비스다. 론칭한 후 10영업일 만에 조회수 10만회를 돌파하며 높은 관심을 받았고, 평균 조회수도 1만회 이상을 기록하며 현재까지 누적 14만회를 넘겼다. 가장 높은 조회수를 기록한 보고서는 지난 5월 15일 발간된 '반짝 회복 뒤 다가올 긴 그림자'로 약 2만8000회를 기록했다.\n골드만삭스 제휴 리서치는 한국투자증권 모바일트레이딩시스템(MTS)과 홈페이지를 통해 무료로 제공되며, 글로벌 거시경제 이슈부터 주요 자산군 전망까지 폭넓게 다룬다.\n한국투자증권의 글로벌 리서치 협업은 이번이 처음은 아니다. 앞서 지난해 3월에는 미국 금융사 스티펄파이낸셜과 손잡고 '슬립리스 인 USA(Sleepless in USA)'를 출시했다.\n유종우 한국투자증권 리서치본부장은 \"해외주식 투자 규모가 사상 최대치를 경신하는 가운데, 정보 격차로 인해 투자 수요가 여전히 일부 종목에 편중돼 있다\"며 \"앞으로도 글로벌 리서치 역량을 강화해 투자자에게 보다 풍부하고 균형 잡힌 정보를 제공하겠다\"고 말했다.\n[김제림 기자]",
        "press": "매일경제",
        "url": "https://n.news.naver.com/mnews/article/009/0005506504",
        "image": "https://imgnews.pstatic.net/image/009/2025/06/10/0005506504_001_20250610174408978.jpg?type=w800",
        "article_preprocessed": "한달새 조회수 14만회 돌파 김성환 한투證 사장 한국투자증권이 지난달 골드만삭스와 제휴해 선보인 '독점 미국 현지 리서치'의 누적 조회수가 14만회를 돌파했다고 10일 밝혔다. 한 달간 발간된 보고서 가운데 서학개미들이 가장 주목한 이슈는 '3분기 중국 경제전망'이었다. '독점 미국 현지 리서치'는 한국투자증권이 글로벌 리서치 차별화를 위해 세계 최대 투자은행 골드만삭스와 손잡고 출시한 서비스다. 론칭한 후 10영업일 만에 조회수 10만회를 돌파하며 높은 관심을 받았고, 평균 조회수도 1만회 이상을 기록하며 현재까지 누적 14만회를 넘겼다. 가장 높은 조회수를 기록한 보고서는 지난 5월 15일 발간된 '반짝 회복 뒤 다가올 긴 그림자'로 약 2만8000회를 기록했다. 한국투자증권의 글로벌 리서치 협업은 이번이 처음은 아니다. 앞서 지난해 3월에는 미국 금융사 스티펄파이낸셜과 손잡고 '슬립리스 인 USA(Sleepless in USA)'를 출시했다.",
        "summary": "한국투자증권이 골드만삭스와 제휴해 선보인 '독점 미국 현지 리서치'의 누적 조회수가 10영업일 만에 10만회를 돌파하며 높은 관심을 받았고, 평균 조회수도 1만회 이상을 기록하며 현재까지 14만회를 넘겼다.",
    },
]

In [45]:
def get_industry_list_from_stocks(stock_list, stock_to_industry):
    if len(stock_list) > 4 or len(stock_list) < 1:
        return []

    return [
        stock_to_industry.get(stock, "")
        for stock in stock_list
        if stock_to_industry.get(stock, "") != ""
    ]

In [46]:
def load_stock_to_industry_map(kospi_desc_csv_path):
    df = pd.read_csv(kospi_desc_csv_path, encoding="cp949")
    return dict(zip(df["종목명"], df["업종명"]))

In [47]:
stock_to_industry = load_stock_to_industry_map('../../automation/db/KRX_KOSPI_DESCRIPTION.csv')

In [48]:
ner_news = []

# 3 뉴스 종목, 업종명 매칭 함수
if len(summarzied_news) != 0:
    for news in summarzied_news:
        news_summary = news["summary"]
        tokens, labels = get_ner_tokens(
            tokenizer_ner, session_ner, news_summary, id2label
        )
        stock_list = extract_ogg_economy(tokens, labels)

        # 여기서 필터링
        stock_list = filter_official_stocks_from_list(
            stock_list, official_stock_set
        )
        news["stock_list"] = stock_list

        # 종목 없거나 너무 많으면 제외
        if len(stock_list) > 4 or len(stock_list) < 1:
            news["stock_list"] = None

        industry_list = get_industry_list_from_stocks(stock_list, stock_to_industry)
        news["industry_list"] = industry_list

        if len(industry_list) < 1:
            news["industry_list"] = None

        ner_news.append(news)

In [51]:
def get_news_deduplicate_by_title(news_list):
    seen_titles = set()
    deduped_news = []

    for news in news_list:
        title = news.get("title")
        if title not in seen_titles:
            seen_titles.add(title)
            deduped_news.append(news)

    return deduped_news

In [52]:
get_news_deduplicate_by_title(ner_news)

[{'news_id': '20250610_08827802',
  'wdate': '2025-06-10 17:45',
  'title': '컴포즈커피 산 필리핀 기업, 노랑통닭도 삼켰다',
  'article': '글로벌 외식업체 졸리비\n인수 우선협상대상자 선정\n연이은 M&A … 韓공략 속도\n필리핀 최대 외식업체 졸리비푸드(이하 졸리비)가 지난해 국내 저가 커피 2위 업체인 컴포즈커피를 인수한 데 이어 올해 하반기 국내 치킨 프랜차이즈 노랑통닭을 운용하는 노랑푸드도 인수할 예정이다.\n10일 투자은행(IB) 업계에 따르면 코스톤아시아와 큐캐피탈파트너스는 노랑푸드 인수 우선협상대상자로 졸리비를 선정하고 이날 오전 양해각서(MOU)를 체결했다.\n인수 주체는 졸리비의 한국법인인 졸리케이이며 조만간 상세 실사에 착수할 예정이다.\n양측은 오는 8월 말까지 실사를 마무리하고 올해 내로 주식매매계약(SPA)을 체결할 전망이다. 예상 매각가는 1000억원대 중반이다.\n졸리비는 큐캐피탈과 코스톤이 보유한 노랑통닭 운영사 노랑푸드 지분 100%를 인수하게 된다. 졸리비는 필리핀에 본사를 둔 글로벌 외식기업이다. 동남아시아를 비롯해 미국·유럽·중국 등 전 세계에서 매장을 운영 중이다.\n졸리비는 국내 사모펀드 운용사인 엘리베이션에쿼티파트너스와 함께 지난해 국내 저가 커피 2위 업체인 컴포즈커피를 4700억원에 인수하며 한국 시장에 처음으로 진출한 바 있다.\n코스톤과 큐캐피탈은 2020년 노랑푸드 지분 100%를 약 700억원에 인수했다.\n두 운용사가 인수한 이후 노랑통닭 실적은 크게 개선됐다. 2019년 말 400여 개였던 가맹점 수는 올해 700개를 넘겼다. 2019년 502억원이던 매출도 2024년 1067억원으로 두 배 넘게 올랐다.\n[나현준 기자 / 정슬기 기자]',
  'press': '매일경제',
  'url': 'https://n.news.naver.com/mnews/article/009/0005506513',
  'image': 'https://img

In [49]:
ner_news

[{'news_id': '20250610_08827802',
  'wdate': '2025-06-10 17:45',
  'title': '컴포즈커피 산 필리핀 기업, 노랑통닭도 삼켰다',
  'article': '글로벌 외식업체 졸리비\n인수 우선협상대상자 선정\n연이은 M&A … 韓공략 속도\n필리핀 최대 외식업체 졸리비푸드(이하 졸리비)가 지난해 국내 저가 커피 2위 업체인 컴포즈커피를 인수한 데 이어 올해 하반기 국내 치킨 프랜차이즈 노랑통닭을 운용하는 노랑푸드도 인수할 예정이다.\n10일 투자은행(IB) 업계에 따르면 코스톤아시아와 큐캐피탈파트너스는 노랑푸드 인수 우선협상대상자로 졸리비를 선정하고 이날 오전 양해각서(MOU)를 체결했다.\n인수 주체는 졸리비의 한국법인인 졸리케이이며 조만간 상세 실사에 착수할 예정이다.\n양측은 오는 8월 말까지 실사를 마무리하고 올해 내로 주식매매계약(SPA)을 체결할 전망이다. 예상 매각가는 1000억원대 중반이다.\n졸리비는 큐캐피탈과 코스톤이 보유한 노랑통닭 운영사 노랑푸드 지분 100%를 인수하게 된다. 졸리비는 필리핀에 본사를 둔 글로벌 외식기업이다. 동남아시아를 비롯해 미국·유럽·중국 등 전 세계에서 매장을 운영 중이다.\n졸리비는 국내 사모펀드 운용사인 엘리베이션에쿼티파트너스와 함께 지난해 국내 저가 커피 2위 업체인 컴포즈커피를 4700억원에 인수하며 한국 시장에 처음으로 진출한 바 있다.\n코스톤과 큐캐피탈은 2020년 노랑푸드 지분 100%를 약 700억원에 인수했다.\n두 운용사가 인수한 이후 노랑통닭 실적은 크게 개선됐다. 2019년 말 400여 개였던 가맹점 수는 올해 700개를 넘겼다. 2019년 502억원이던 매출도 2024년 1067억원으로 두 배 넘게 올랐다.\n[나현준 기자 / 정슬기 기자]',
  'press': '매일경제',
  'url': 'https://n.news.naver.com/mnews/article/009/0005506513',
  'image': 'https://img

In [57]:
import pandas as pd

df = pd.read_csv('../../db/news_2023_2025_summarized_view.csv')
df['article'][20]

'사진=한경DB\n키움증권은 23일 삼성바이오로직스에 대해 인적분할을 추진하는 건 본격적인 신약 개발에 나서겠다는 의미하고 해석했다.\n삼성바이오로직스는 인적분할을 통해 의약품 위탁 개발·생산(CDMO) 부문은 존속법인에 남기고, 신설법인인 삼성에피스홀딩스에 삼성바이오에피스 지분 100%를 승계하도록 하는 방안을 추진하겠다고 공시했다.\n이에 대해 허혜민 키움증권 연구원은 “삼성그룹이 (삼성에피스홀딩스를 통해) 본격적으로 신약 개발을 시작한다는 의미로 해석된다”고 말했다. 이어 “삼성바이오로직스는 글로벌 파트너십 확대를 위한 영업활동이 수월해질 것”이라며 “분할된 신설법인은 투자지주회사로 신성장 동력 발굴과 연구·개발(R&D) 및 기업 인수·합병(M&A)를 통해 적극적인 성장을 추진할 것”이라고 내다봤다.\n이번 분할 추진이 단기적으로 펀더멘털에 미치는 영향은 미미할 것으로 분석됐다.\n다만 허 연구원은 “최근 인적분할 업체들의 분할상장 전 주가 수익률이 대체로 상승 흐름을 나타냈다”며 “이후 각 사의 펀더멘털, 수급, 모멘텀에 따라 변동했다”고 전했다.'

In [13]:
df = df[['news_id', 'summary', 'stock_list', 'industry_list']]
df.head()

,news_id,summary,stock_list,industry_list
0,20250523_0002,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 국내 대표 전자결제사업자인 카...,['카카오페이'],['금융 지원 서비스업']
1,20250523_0004,세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자(COO)인 존 월드...,['삼성전자'],['통신 및 방송 장비 제조업']
2,20250523_0007,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 카카오페이가 SSG닷컴 쓱페이...,['카카오페이'],['금융 지원 서비스업']
3,20250523_0010,23일 효성중공업은 조 회장이 상속세 재원을 마련하기 위해 시간 외 매매로 효성중 ...,['효성중공업'],"['전동기, 발전기 및 전기 변환 · 공급 · 제어 장치 제조업']"
4,20250523_0011,카카오페이가 신세계 이마트 측에서 쓱페이·스마일페이 인수를 추진하고 나선 건 국내 ...,['카카오페이'],['금융 지원 서비스업']


In [14]:
df.to_csv('../../db/news_2023_2025_metadata.csv', index=False)

In [ ]:
import redis

redis_client = redis.Redis(
    host="100.68.180.102", port=6379, password="q1w2e3r4!@#", decode_responses=True
)
pubsub = redis_client.pubsub()
pubsub.subscribe("news_channel")

for message in pubsub.listen():
    if message["type"] == "message":
        print(message['data'])

{"news_id": "20250610_2777", "wdate": "2025-06-10 17:45", "title": "카카오그룹주 124급등…경고장 날린 JP412412모간DDDDD", "article": "카카오페이 한달 95% 급등\nJP모간 \"정당화하기 어렵다\"\n최근 카카오그룹주가 급등한 데 대해 외국계 투자은행(IB) JP모간이 “정당화하기 어려운 주가 급등세”라고 평가절하했다.\n10일 한국거래소에 따르면 최근 한 달간 카카오페이 주가는 95.24% 뛰었다. 같은 기간 카카오가 35.32%, 카카오뱅크는 24.89% 올랐다. 카카오페이를 필두로 카카오그룹주가 급등한 건 이재명 정부 출범과 함께 원화 스테이블코인 도입 논의가 본격화할 것이란 기대가 커졌기 때문이란 분석이 나온다. 지난 6일 초대 대통령실 정책실장으로 원화 스테이블코인 도입 필요성을 주장해 온 김용범 전 기획재정부 1차관이 선임되며 주가 상승에 탄력이 붙었다.\n그러나 JP모간은 이날 “카카오페이 급등은 근본적으로 정당화하기 어렵다”고 지적했다. JP모간은 “원화 기반 스테이블코인 정책 수혜주로 카카오페이를 단정하기엔 시기상조”라며 “스테이블코인 도입과 관련한 불확실성도 여전히 크다”고 덧붙였다.\n지역화폐 바우처 정책의 효과도 미미할 것으로 봤다. JP모간은 “지역화폐를 통한 소비 진작 정책은 일회성”이라며 “카카오페이의 올해 영업이익을 30억원가량 늘리는 데 그칠 것”이라고 말했다. JP모간은 “최근 카카오그룹주 상승세는 실적이 아니라 기대만으로 급등한 2022년 말~2023년 초와 비슷하다”고 했다.\n이날 카카오페이는 15.96% 상승한 5만7400원에 거래를 마쳤다. 카카오와 카카오뱅크는 각각 2.72%, 2.74% 하락했다.", "press": "한국경제2", "url": "https://n.news.naver.com/mnews/article/015/0005142865123", "image": "https://imgnews.pstatic.net/image/015/202

KeyboardInterrupt: 

In [10]:
import redis
import json

redis_client = redis.Redis(
    host="43.200.17.139", port=6379, password="q1w2e3r4!@#", decode_responses=True
)

if not redis_client.ping():
    print("Redis 서버에 연결할 수 없습니다.")

channel = "news-channel"
data = {
    "news_id":123,
    "wdate": "2025-06-10T17:45:00",
    "title": "컴포즈커피 산 필리핀 기업, 노랑통닭도 삼켰다",
    "article": 'faejfijsfjoaifjaijfo',
    "press": "매일경제",
    "url": 'ddd@naver.com',
    "image": 'https://imgnews.pstatic.net/image/009/2025/06/10/0005506513_001_20250610174509011.jpg?type=w800',
}
message = json.dumps(data, ensure_ascii=False)
redis_client.publish(channel, message)


ConnectionError: Error 10060 connecting to 43.200.17.139:6379. 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다.

In [13]:
wdate_str = "2025-06-10 17:45"
from datetime import datetime
wdate = datetime.strptime(wdate_str, "%Y-%m-%d %H:%M").isoformat()

In [14]:
wdate

'2025-06-10T17:45:00'